In [1]:
skip_download = True  # If True, skip the download step
skip_transform = True  # If True, skip the transform step
skip_tracing = True  # If True, skip the tracing step

In [2]:
import os

if not skip_tracing:
    os.environ["LANGCHAIN_TRACING_V2"] = "true"
    os.environ["LANGCHAIN_PROJECT"] = "Tx Generator"

In [3]:
from case_code.code_downloader import CodeDownloader, get_metadata
from case_code.code_loader import CodeLoader
from case_code.code_transformer import transform

from models.case import BatchCase
from utils.model_selector import get_chat_model

In [4]:
if not skip_download:
    downloader = CodeDownloader()
    downloaded_count = downloader.download()

In [5]:
metadata = get_metadata()
print(f"{'Total files downloaded:':<24} {metadata['total_downloads']}")
print(f"{'Total cases:':<24} {metadata['total_cases']}")

Total files downloaded:  125
Total cases:             110


In [6]:
import pandas as pd

if not skip_transform:
    # Transform the code and process the results
    loader = CodeLoader()
    transformed_data = await transform(loader=loader)
    # Save the transformed data to a file
    data = [(key.value, value) for key, value in transformed_data.items()]
    df = pd.DataFrame(data, columns=["Title", "Count"])
    print(df.to_string(index=False))

In [7]:
from case_code.code_transformer import get_transformation_result

df = get_transformation_result()
filtered_df = df.loc[df["transformable"] == False]
filtered_df

,id,transformable


In [8]:
from case import get_retriever

retriever = get_retriever()

In [9]:
retriever.invoke("Stake ETH with Lido and deposit to Eigenpie")

[Document(metadata={'case_id': 'eigenpie_steth', 'seq_num': 5, 'source': 'eigenpie/index.ts', 'steps': "[{'description': 'Stake ETH to Lido'}, {'description': 'Approve stETH to Eigenpie'}, {'description': 'Stake stETH to Eigenpie'}]", 'total_steps': 3}, page_content='A batch case for staking ETH to Lido and then to Eigenpie.'),
 Document(metadata={'case_id': 'stake_eth', 'seq_num': 79, 'source': 'stake-eth/stake.ts', 'steps': "[{'description': 'Stake into Lido'}, {'description': 'Stake into RocketPool'}, {'description': 'Swap ETH to staked token using 1inch for Coinbase'}, {'description': 'Stake into StakeWise'}, {'description': 'Stake into Stader'}, {'description': 'Stake into EtherFi'}, {'description': 'Stake into Renzo'}, {'description': 'Stake into FraxFinance'}, {'description': 'Stake into Swell'}, {'description': 'Stake into RedactedFinance'}]", 'total_steps': 14}, page_content='Stake ETH into various staking protocols including Lido, RocketPool, and more.'),
 Document(metadata={

In [10]:
retriever.invoke("Swap 10 USDC to USDT")

[Document(metadata={'case_id': 'etf_base_meme_top10', 'seq_num': 110, 'source': 'etf-basememe-top10/index.ts', 'steps': "[{'description': 'Approve USDC for 1inch.'}, {'description': 'Swap USDC to Top1 token.'}, {'description': 'Swap USDC to Top2 token.'}, {'description': 'Swap USDC to Top3 token.'}, {'description': 'Swap USDC to Top4 token.'}, {'description': 'Swap USDC to Top5 token.'}, {'description': 'Swap USDC to Top6 token.'}, {'description': 'Swap USDC to Top7 token.'}, {'description': 'Swap USDC to Top8 token.'}, {'description': 'Swap USDC to Top9 token.'}, {'description': 'Swap USDC to Top10 token.'}]", 'total_steps': 11}, page_content='Batch case for swapping USDC to the top 10 tokens based on market cap using 1inch.'),
 Document(metadata={'case_id': 'etf_ethereum_top10', 'seq_num': 34, 'source': 'etf-ethereum-top10/index.ts', 'steps': "[{'description': 'Approve USDT for 1inch.'}, {'description': 'Swap USDT to Top1 token.'}, {'description': 'Swap USDT to Top2 token.'}, {'descr

In [11]:
from langchain_core.runnables import RunnablePassthrough
from langchain.prompts import ChatPromptTemplate


template = """Generate a structure output based only on the following context:
{context}
Question: {question}"""

prompt = ChatPromptTemplate.from_template(template)

retriever = get_retriever()

model = get_chat_model()


def format_docs(docs):
    return "\n\n".join(str(doc.metadata) for doc in docs)


# Chain
rag_chain = (
    {"context": retriever | format_docs, "question": RunnablePassthrough()}
    | prompt
    | model.with_structured_output(BatchCase)
)

In [12]:
question = "Stake 0.03 ETH with Lido and deposit to Eigenpie"
result = rag_chain.invoke(question)
print(result.json())

{"id": "eigenpie_steth", "description": "Stake ETH with Lido and deposit to Eigenpie", "total_steps": 3, "steps": [{"description": "Stake ETH to Lido"}, {"description": "Approve stETH to Eigenpie"}, {"description": "Stake stETH to Eigenpie"}]}


In [13]:
question = "Stake 2 ETH with Lido and deposit to Eigenpie"
result = rag_chain.invoke(question)
print(result.json())

{"id": "stake_2_eth_lido_eigenpie", "description": "Stake 2 ETH with Lido and deposit to Eigenpie", "total_steps": 3, "steps": [{"description": "Stake ETH to Lido"}, {"description": "Approve stETH to Eigenpie"}, {"description": "Stake stETH to Eigenpie"}]}


In [14]:
question = "Swap 100 USDT for USDC on Uniswap"
result = rag_chain.invoke(question)
print(result.json())

{"id": "swap_usdt_for_usdc", "description": "Swap 100 USDT for USDC on Uniswap.", "total_steps": 2, "steps": [{"description": "Approve USDT for Uniswap."}, {"description": "Swap 100 USDT for USDC."}]}


In [15]:
question = "Swap 0.001 ETH for USDC on Uniswap"
result = rag_chain.invoke(question)
print(result.json())

{"id": "swap_eth_for_usdc_uniswap", "description": "Swap 0.001 ETH for USDC on Uniswap", "total_steps": 1, "steps": [{"description": "Swap 0.001 ETH for USDC on Uniswap"}]}


In [16]:
question = """Yield Farming with Multiple Protocols
	Step 1: Deposit ETH into a AAVE to earn interest.
	Step 2: Use aToken (aETH) as collateral to borrow DAI.
	Step 3: Deposit the borrowed DAI into a Yearn Finance.
	Step 4: Use the Yearn vault tokens in a liquidity pool on Uniswap to earn trading fees."""
result = rag_chain.invoke(question)
print(result.json())

{"id": "yield_farming_multiple_protocols", "description": "Yield farming strategy involving multiple protocols to maximize returns through deposits, borrowing, and liquidity provision.", "total_steps": 4, "steps": [{"description": "Deposit ETH into AAVE to earn interest."}, {"description": "Use aToken (aETH) as collateral to borrow DAI."}, {"description": "Deposit the borrowed DAI into Yearn Finance."}, {"description": "Use the Yearn vault tokens in a liquidity pool on Uniswap to earn trading fees."}]}
